In [ ]:
import os
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_openml

In [ ]:
mnist = fetch_openml('mnist_784', version=1)
mnist.keys()
# DESCR: 데이터셋 설명
# data: 샘플이 하나의 행, 특성이 하나의 열로 구성된 배열
# target: 레이블 배열

In [ ]:
X, y = mnist['data'], mnist['target']
X.shape, y.shape
# 28x28 = 784개의 feature

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

some_digit = X[0]
some_digit_image = some_digit.reshape(28, 28)

plt.imshow(some_digit_image, cmap='binary')
plt.axis('off')
plt.show()

In [ ]:
y[0]

In [ ]:
# label이 문자열이다. 대부분 머신러닝 알고리즘은 숫자를 기대하므로 y를 정수로 변환.
y = y.astype(np.uint8)

In [ ]:
# MNIST의 훈련 세트는 이미 섞여 있어서 모든 교차 검증 폴드를 비슷하게 만듬.
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [ ]:
# 5만을 식별하는 이진 분류기
y_train_5 = (y_train == 5)
y_test_5 = (y_test == 5)

In [ ]:
from sklearn.linear_model import SGDClassifier
# 확률적 경사 하강법 (Stochastic GD)

sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_5)
sgd_clf.predict([some_digit])

In [ ]:
# cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

skfolds = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)

# 3 fold
for train_index, test_index in skfolds.split(X_train, y_train_5):
    clone_clf = clone(sgd_clf)
    X_train_folds = X_train[train_index]
    y_train_folds = y_train_5[train_index]
    X_test_fold = X_train[test_index]
    y_test_fold = y_train_5[test_index]

    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print(n_correct / len(y_pred))

In [ ]:
from sklearn.model_selection import cross_val_score

cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring='accuracy')

In [ ]:
from sklearn.base import BaseEstimator

class Never5Classifier(BaseEstimator):
    def fit(self, X, y=None):
        return self
    
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

never_5_clf = Never5Classifier()
cross_val_score(never_5_clf, X_train, y_train_5, cv=3, scoring='accuracy')

# 모든 이미지를 '5 아님'으로 분류했을때 정확도 90% 나옴
# accuracy를 성능 측정 지표로 선호하지 않는 이유
# 불균형한 데이터셋(어떤 클래스가 다른 것보다 월등히 많은 겅우)을 다룰때 더욱 심하다

In [ ]:
# 오차 행렬 (confusion matrix)
# 클래스 A를 클래스 B로 잘못 분류된 횟수를 세는 것.
# 분류기가 숫자 5의 이미지를 3으로 잘못 분류한 횟수 -> 오차 행렬의 5행 3열

# 먼저 실제 타깃과 비교를 위한 예측값 생성
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)

# 오차 행렬 생성
# 행=실제 클래스, 열=예측한 클래스
from sklearn.metrics import confusion_matrix

confusion_matrix(y_train_5, y_train_pred)
# 첫번째 행 '5 아님' 이미지를 '5 아님'으로 정확히 분류: 53057개, 나머지 1522개는 '5'라고 잘못 분류
# 두번째 행 '5' 이미지를 '5 아님'으로 잘못 분류: 1325개, 나머지 4096개는 정확히 '5'라고 분류
#                    예측
#               음성        양성
#       음성   53892        687
# 실제                             ㅅ
#       양성   1891         3530   | 정밀도
#                    <-재현율

In [ ]:
# 완벽한 분류기의 경우 진짜 양성과 진짜 음성만 있을 것이므로 오차 행렬의 주대각선만 0이 아님
y_train_perfect_predictions = y_train_5
confusion_matrix(y_train_5, y_train_perfect_predictions)

In [ ]:
from sklearn.metrics import precision_score, recall_score

# 정밀도 (precision)
print(precision_score(y_train_5, y_train_pred))
# 재현율 (recall)
print(recall_score(y_train_5, y_train_pred))

In [ ]:
# F1 score = 정밀도와 재현율의 조화 평균(harmonic mean)
from sklearn.metrics import f1_score

f1_score(y_train_5, y_train_pred)
# 정밀도를 올리면 재현율이 줄고 그 반대도 마찬가지. 정밀도/재현율 트레이드오프

In [ ]:
y_score = sgd_clf.decision_function([some_digit])
print(y_score)
threshold = 0
y_some_digit_pred = (y_score > threshold)
y_some_digit_pred

# SGDClassifier의 임곗값이 0이므로 predict와 같은 결과 반환

In [ ]:
threshold = 8000
y_some_digit_pred = (y_score > threshold)
y_some_digit_pred

In [ ]:
# 모든 샘플의 점수를 구함 (예측 결과가 아닌 결정 점수 반환)
y_scores = cross_val_predict(sgd_clf, X_train, y_train_5,
                            cv=3, method='decision_function')

In [ ]:
# 모든 임계값에 대해 정밀도와 재현율을 계산
from sklearn.metrics import precision_recall_curve

precisions, recalls, thresholds = precision_recall_curve(y_train_5, y_scores)

def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], 'b--', label='정밀도')
    plt.plot(thresholds, recalls[:-1], 'g-', label='재현율')

plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.show()

In [ ]:
# argmax는 최댓값의 첫번째 인덱스를 반환. 여기서는 첫번째 True 값을 의미.
threshold_90_precision = thresholds[np.argmax(precisions >= 0.90)]
print(threshold_90_precision)

# 임계값 설정 (점수가 3370보다 높은것들)
y_train_pred_90 = (y_scores >= threshold_90_precision)
print(y_train_pred_90)

print(precision_score(y_train_5, y_train_pred_90))
print(recall_score(y_train_5, y_train_pred_90))

In [ ]:
def plot_precision_vs_recall(precisions, recalls):
    plt.plot(recalls, precisions, "b-", linewidth=2)
    plt.xlabel("Recall", fontsize=16)
    plt.ylabel("Precision", fontsize=16)
    plt.axis([0, 1, 0, 1])
    plt.grid(True)

plot_precision_vs_recall(precisions, recalls)
plt.show()
# 곡선이 오른쪽 위에 가까워질수록 좋음

In [ ]:
from sklearn.metrics import roc_curve

# fpr = 1 - tnr
# tpr = 재현율
fpr, tpr, thresholds = roc_curve(y_train_5, y_scores)

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate (Fall-Out)', fontsize=16) # Not shown
    plt.ylabel('True Positive Rate (Recall)', fontsize=16)    # Not shown
    plt.grid(True)                                            # Not shown

plot_roc_curve(fpr, tpr)
plt.show()
# 좋은 분류기는 점선에서 최대한 멀리 떨어져 있어야함 (왼쪽 위)
# 점선은 완전 랜덤 분류기

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_train_5, y_scores)
# 완벽한 분류기는 auc=1 (accuracy 아님)
# 완전 랜덤 분류기는 auc=0.5 (이진 분류기라서)

양성 클래스가 드물거나 거짓 음성보다 거짓 양성이 더 중요할때 PR 곡선을 사용. (정밀도/재현율)
그렇지 않으면 ROC 사용.

예를 들어 방금전 ROC 곡선이 잘 나와서 좋은 분류기라고 생각할 수 있는데 이는 음성(5아님)에 비해 양성(5)이 크게 적기 때문이다.
이와는 다르게 PR 곡선은 분류기의 성능 개선 여지가 얼마나 되는지(오른쪽 위 모서리) 잘 보여준다.


In [ ]:
# 사이킷런은 일반적으로 decision_function() 또는 predict_proba() 메서드 둘 다 또는 둘 중 하나를 가지고있다.
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(random_state=42)

y_probas_forest = cross_val_predict(forest_clf, X_train, y_train_5,
                                    cv=3, method='predict_proba')
# predict_proba() 메서느는 샘플이 행, 클래스가 열이고
# 샘플이 주어진 클래스에 속할 확률을 담은 배열을 반환
# (어떤 이미지가 5일 확률 70%)
print(y_probas_forest)

In [ ]:
# roc_curve()는 레이블과 점수를 기대한다. 하지만 점수 대신 클래스 확률을 전달 할 수 있다.
y_scores_forest = y_probas_forest[:, 1] # 양성 클래스에 대한 확률을 점수로 사용
fpr_forest, tpr_forest, thresholds_forest = roc_curve(y_train_5, y_scores_forest)

In [ ]:
plt.plot(fpr, tpr, "b:", linewidth=2, label="SGD") 
plot_roc_curve(fpr_forest, tpr_forest, "Random Forest") 
plt.grid(True)
plt.legend(loc="lower right")
plt.show()

In [ ]:
print(roc_auc_score(y_train_5, y_scores_forest))

y_train_pred_forest = cross_val_predict(forest_clf, X_train, y_train_5, cv=3)
print(precision_score(y_train_5, y_train_pred_forest))
print(recall_score(y_train_5, y_train_pred_forest))

다중 분류

In [ ]:
# OvR : 특정 숫자 하나만 구분하는 숫자별 이진 분류기 10개(0~9)
# OvO : 0과 1 구별, 1과 2 구별 등 각 숫자의 조합마다 이진 분류기를 훈련. (OvO 장점, 각 분류기의 훈련에 두 클래스에 해당하는 샘플만 필요)
# SVM 빼고는 대부분 OvR 선호
# 사이킷런은 알고리즘에 따라 자동으로 OvO 또는 OvR 실행
from sklearn.svm import SVC

svm_clf = SVC()
svm_clf.fit(X_train, y_train) # y_train_5가 아니고 y_train 사용
svm_clf.predict([some_digit])
# 내부에서 사이킷런이 OvO 전략을 사용해 45개의 이진 분류기를 훈련시키고 각각의 결정 점수를 얻어 점수가 가장 높은 클래스 선택.

In [ ]:
some_digit_scores = svm_clf.decision_function([some_digit])
print(some_digit_scores)

print(np.argmax(some_digit_scores))

print(svm_clf.classes_)
print(svm_clf.classes_[5])

In [ ]:
# 강제로 OvO나 OvR을 사용하는 법
from sklearn.multiclass import OneVsRestClassifier

ovr_clf = OneVsRestClassifier(SVC())
ovr_clf.fit(X_train, y_train)
ovr_clf.predict([some_digit])